In [1]:
#Iván Andrés Trujillo Abella 
#ivantrujillo1229@gmail.com
#Consolidate Thesis

In [2]:
## Own functions
def error_in_date(x, format_date= "%Y-%m-%d"):
    try:
        datetime.strptime(x,format_date)
        return False
    except:
        return True

def clean_date(x):
    return pd.to_datetime(x[0:10], format= "%Y-%m-%d")    
    
def cleantime(x):
    if type(x)==str:
        return x.replace("00:00:00", "").replace(" ", "")
    else:
        return x

def top_cat(k, cat, df):
    otro = []
    cat_relative_dict =  df[cat].value_counts(normalize=True).to_dict()
    for cat in cat_relative_dict:
        if cat_relative_dict[cat]<=k:
            otro.append(cat)
    return otro


def bankyear(df_):
    df = df_.copy()
    years = [2016, 2017, 2018, 2019]
    df['bankrupt-year'] = 0
    for firm in df.index:
        unchanged = True
        for year in years:
            if df.loc[firm, 'evento'+str(year)]==1 and unchanged==True:
                df.loc[firm,'bankrupt-year']= year
                unchanged = False
    return df['bankrupt-year']



def new_firms(current_df, next_df, key):
    firm_first =  set(current_df[key].unique()) -  set(next_df[key].unique())
    firm_second =  set(next_df[key].unique()) -  set(current_df[key].unique())
    return pd.concat([current_df[current_df[key].isin(firm_first)] , next_df[next_df[key].isin(firm_second)]])


# A function
def varInEvent(time_base, prefix, var_time_event, df_, lag=1):
    df = df_.copy()
    df['var*'] = np.nan
    for firm in df.index:
        time_ocurrence = df.loc[firm, var_time_event]
        if time_ocurrence - lag >= time_base:
            df.loc[firm, 'var*'] = df.loc[firm, prefix + str(time_ocurrence-lag)]
    return df['var*']


In [3]:
cd C:\Users\iván\Desktop\Thesis\Thesis-bankrupt\Datasets

C:\Users\iván\Desktop\Thesis\Thesis-bankrupt\Datasets


In [4]:
import pandas as pd
import numpy as np

In [5]:
# Loading datasets
dfs = {}
years = [2016,2017,2018,2019]
for year in years:
  dfs['df'+str(year)]  = pd.read_excel('Cáratula-Pymes-'+str(year)+'.xlsx')

In [6]:
for year in years:
  print('#'*10)
  print(dfs['df'+str(year)].shape , year)
  dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Estado actual']!='EN ETAPA PREOPERATIVA']
  print(dfs['df'+str(year)].shape , year)
  dfs['df'+str(year)]['Fecha'] = pd.to_datetime( dfs['df'+str(year)][ 'Fecha de Corte'], format= "%Y-%m-%d")
  dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Fecha'].dt.month==12] # there is duplicates given present the information two or more times..
  print(dfs['df'+str(year)].shape , year)
  dfs['df'+str(year)] =  dfs['df'+str(year)][-pd.isnull(dfs['df'+str(year)]['NIT'])]
  print(dfs['df'+str(year)].shape , year)
  dfs['df'+str(year)]['evento'+str(year)]  = np.where(dfs['df'+str(year)]['Estado actual']!='ACTIVA',1 ,0)
  dfs['df'+str(year)] = dfs['df'+str(year)][['evento'+str(year), 'NIT']]
  print(dfs['df'+str(year)]['NIT'].duplicated().sum())
  print(dfs['df'+str(year)].shape , year)

##########
(17799, 31) 2016
(17723, 31) 2016
(17701, 32) 2016
(17701, 32) 2016
0
(17701, 2) 2016
##########
(14696, 32) 2017
(14656, 32) 2017
(14639, 33) 2017
(14638, 33) 2017
0
(14638, 2) 2017
##########
(14275, 34) 2018
(14229, 34) 2018
(14212, 35) 2018
(14210, 35) 2018
1
(14210, 2) 2018
##########
(19185, 36) 2019
(19140, 36) 2019
(19128, 37) 2019
(19125, 37) 2019
0
(19125, 2) 2019


In [7]:
dfs['df2019']['evento2019'].value_counts() # Here there are 876 firms,  but since declarate the process.

evento2019
0    18249
1      876
Name: count, dtype: int64

In [8]:
for year in range(2016,2020):
    print(dfs['df'+str(year)]['evento'+str(year)].value_counts())

evento2016
0    17193
1      508
Name: count, dtype: int64
evento2017
0    14019
1      619
Name: count, dtype: int64
evento2018
0    13445
1      765
Name: count, dtype: int64
evento2019
0    18249
1      876
Name: count, dtype: int64


In [9]:
full = dfs['df2016'].copy()
for year in range(2017,2020):
    clients= new_firms(full,dfs['df'+str(year)], 'NIT')
    full = full.merge(dfs['df'+str(year)], on='NIT', how='inner', suffixes=(None, 'to_remove'))
    duplicated = [x for x in full.columns if 'to_remove' in x]
    full.drop(columns=duplicated, inplace=True)
    full = pd.concat([full, clients])

In [10]:
full['NIT'].duplicated().sum()

1

In [11]:
full['evento2019'].value_counts()

evento2019
0.0    18250
1.0      876
Name: count, dtype: int64

In [12]:
full = full[-full['NIT'].duplicated()].reset_index(drop=True)
full['time-event'] =bankyear(full)
full['time-event'].value_counts()

time-event
0       25931
2016      508
2019      282
2018      257
2017      232
Name: count, dtype: int64

In [13]:
## Draw firmst that enter to process in 2016 to avoid bias in indicators...
full = full[full['evento2016']!=1]
print(full['time-event'].value_counts())

time-event
0       25931
2019      282
2018      257
2017      232
Name: count, dtype: int64


In [14]:
full['event'] = (full['time-event']!=0).astype('int')

In [15]:
full['event'].value_counts()

event
0    25931
1      771
Name: count, dtype: int64

In [16]:
ERIS={}
EFSS={}
for year in range(2016,2019):
        #loops in ERI
    ERIS['eri'+str(year)] = pd.read_excel('ERI-Pymes-'+str(year)+'.xlsx')
    ERIS['eri'+str(year)]['Fecha'] = pd.to_datetime( ERIS['eri'+str(year)][ 'Fecha de Corte'], format= "%Y-%m-%d")
    ERIS['eri'+str(year)] = ERIS['eri'+str(year)][ERIS['eri'+str(year)]['Fecha'].dt.month==12]
    if year==2016:
        ERIS['eri2016'] = ERIS['eri2016'][pd.to_datetime( ERIS['eri2016'][ 'Periodo'], format= "%Y-%m-%d").dt.year==2016]
        ERIS['eri2016'] = ERIS['eri2016'][pd.to_datetime( ERIS['eri2016'][ 'Periodo'], format= "%Y-%m-%d").dt.month==12]
        ERIS['eri2016'] = ERIS['eri2016'][pd.to_datetime( ERIS['eri2016'][ 'Periodo'], format= "%Y-%m-%d").dt.day==31]
    else:
        ERIS['eri'+str(year)] = ERIS['eri'+str(year)][ERIS['eri'+str(year)]['Periodo']=='Periodo Actual']
        ERIS['eri'+str(year)] = ERIS['eri'+str(year)][pd.to_datetime(ERIS['eri'+str(year)]['Fecha de Corte'], format= "%Y-%m-%d").dt.day==31]

    #ERIS['eri'+str(year)][-ERIS['eri'+str(year)]['NIT'].duplicated()].reset_index(drop=True)

    # loops in efs
    EFSS['efs'+str(year)] = pd.read_excel('ESF-Pymes-'+str(year)+'.xlsx')
    if year == 2016:
        EFSS['efs2016'] =EFSS['efs2016'][EFSS['efs2016']['Periodo'].apply(lambda x : clean_date(x)).dt.year==2016]
        EFSS['efs2016'] =EFSS['efs2016'][EFSS['efs2016']['Periodo'].apply(lambda x : clean_date(x)).dt.month==12]
        EFSS['efs2016'] =EFSS['efs2016'][EFSS['efs2016']['Periodo'].apply(lambda x : clean_date(x)).dt.day==31]
    else:
        EFSS['efs'+str(year)] = EFSS['efs'+str(year)][EFSS['efs'+str(year)]['Periodo']=='Periodo Actual']
        EFSS['efs'+str(year)] = EFSS['efs'+str(year)][pd.to_datetime(EFSS['efs'+str(year)]['Fecha de Corte'], format= "%Y-%m-%d").dt.month==12]
        EFSS['efs'+str(year)] = EFSS['efs'+str(year)][pd.to_datetime(EFSS['efs'+str(year)]['Fecha de Corte'], format= "%Y-%m-%d").dt.day==31]

## Lines to debug...
#EFSS['efs2018']['NIT'].duplicated(keep=False).sum()
#EFSS['efs2018'][EFSS['efs2018']['NIT'].duplicated(keep=False)].to_excel('2018_review.xlsx')

In [37]:
# We can integrate both statements...


FULLS = {}
for year in range(2016,2019):
    # Number of fimrs by year in each statement!
    print('--'*10)
    print(year,EFSS['efs'+str(year)].shape[0] ,  ERIS['eri'+str(year)].shape[0])
    FULLS['full'+str(year)] = EFSS['efs'+str(year)].merge(ERIS['eri'+str(year)], on='NIT', how='outer',
                                                          indicator = True, suffixes=(None, 'to_remove') )
    duplicated = 
    print('--'*10)
    print(FULLS['full'+str(year)]['_merge'].value_counts(dropna=False))

--------------------
2016 17777 17777
--------------------
_merge
both          17777
left_only         0
right_only        0
Name: count, dtype: int64
--------------------
2017 14679 14679
--------------------
_merge
both          14679
left_only         0
right_only        0
Name: count, dtype: int64
--------------------
2018 14257 14257
--------------------
_merge
both          14257
left_only         0
right_only        0
Name: count, dtype: int64


In [39]:
FULLS['full2016'].columns

Index(['punto de entrada_x', 'formulario_x', 'NIT',
       'Razón social de la sociedad_x', 'Fecha de Corte_x',
       'Clasificación Industrial Internacional Uniforme Versión 4 A.C_x',
       'Tipo societario_x',
       'Dirección de notificación judicial registrada en Cámara de Comercio_x',
       'Departamento de la dirección del domicilio_x',
       'Ciudad de la dirección del domicilio_x', 'Periodo_x',
       'Efectivo y equivalentes al efectivo',
       'Cuentas comerciales por cobrar y otras cuentas por cobrar corrientes',
       'Inventarios corrientes', 'Activos por impuestos corrientes corriente',
       'Activos biológicos corrientes al costo menos depreciación acumulada y deterioro de valor',
       'Activos biológicos corrientes al valor razonable',
       'Otros activos financieros corrientes',
       'Otros activos no financieros corrientes',
       'Activos corrientes distintos al efectivo pignorados como garantía colateral para las que el receptor de transferencias tie

In [ ]:
'Ingreso (gasto) por impuestos', 'Ganancia bruta', 